# Deploy your model on a SageMaker Endpoint and predict

## Setup environment

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

role = sagemaker.get_execution_role()
model_artefact = '<your-model-s3-path>'

## Create endpoint

In [ ]:
class SentimentAnalysis(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

model = PyTorchModel(model_data=model_artefact,
                   name=name_from_base('bert-model'),
                   role=role, 
                   entry_point='predict_endpoint.py',
                   source_dir='source_dir',
                   framework_version='1.5.0',
                   predictor_cls=SentimentAnalysis)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

## Predict

In [ ]:
test_data = {"text": "I love completing my todos! Best app ever!!!"}
print(test_data)

In [ ]:
prediction = predictor.predict(test_data)

In [ ]:
print(f'Review text: {test_data}')
print(f'Sentiment  : {prediction}')